$\newcommand{\To}{\Rightarrow}$

In [2]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [15]:
from kernel.type import boolT
from kernel.term import Var
from kernel.thm import Thm
from logic import basic
from logic.logic import conj
from syntax import printer

thy = basic.loadTheory('logic_base')

## Propositional logic

In this section, we describe proofs in propositional logic. Propositional logic has four operators: conjunction ($\wedge$), disjunction ($\vee$), implication ($\to$), and negation ($\neg$). Rules for implication have already been discussed (primitive deduction rules `implies_intr` and `implies_elim`). We will now discuss rules for the other three operators in turn.

The introduction and destruction rules for conjunction are `conjI`, `conjD1` and `conjD2`:

In [9]:
conjI = thy.get_theorem('conjI')
print("conjI:", printer.print_thm(thy, conjI, unicode=True))

conjD1 = thy.get_theorem('conjD1')
print("conjD1:", printer.print_thm(thy, conjD1, unicode=True))

conjD2 = thy.get_theorem('conjD2')
print("conjD2:", printer.print_thm(thy, conjD2, unicode=True))

conjI: ⊢ A ⟶ B ⟶ A ∧ B
conjD1: ⊢ A ∧ B ⟶ A
conjD2: ⊢ A ∧ B ⟶ B


We consider these three theorems to be *axioms* that form the fundamental assumptions about conjunction (it is also possible to define conjunction satisfying these axioms in higher-order logic. This is omitted for simplicity). `conjI` states how a conjunction theorem can be proved. `conjD1` and `conjD2` shows how to make use of conjunction theorems.

We now give some examples for working with conjunction.

#### Example:

Prove $A \wedge B \to B \wedge A$.

#### Solution:

0. $A \wedge B \vdash A \wedge B$ by assume $A \wedge B$.
1. $\vdash A \wedge B \to A$ by theorem conjD1.
2. $A \wedge B \vdash A$ by implies_elim from 1, 0.
3. $\vdash A \wedge B \to B$ by theorem conjD2.
4. $A \wedge B \vdash B$ by implies_elim from 3, 0.
5. $\vdash A \to B \to A \wedge B$ by theorem conjI.
6. $\vdash B \to A \to B \wedge A$ by substitution {A: B, B: A} from 5.
7. $A \wedge B \vdash A \to B \wedge A$ by implies_elim 6, 4.
8. $A \wedge B \vdash B \wedge A$ by implies_elim 7, 2.
9. $\vdash A \wedge B \to B \wedge A$ by implies_intr $A \wedge B$ from 8.

In [19]:
A = Var("A", boolT)
B = Var("B", boolT)
th0 = Thm.assume(conj(A, B))
th1 = thy.get_theorem('conjD1')
th2 = Thm.implies_elim(th1, th0)
th3 = thy.get_theorem('conjD2')
th4 = Thm.implies_elim(th3, th0)
th5 = thy.get_theorem('conjI')
th6 = Thm.substitution({"A": B, "B": A}, th5)
th7 = Thm.implies_elim(th6, th4)
th8 = Thm.implies_elim(th7, th2)
th9 = Thm.implies_intr(conj(A, B), th8)
print(printer.print_thm(thy, th9, unicode=True))

⊢ A ∧ B ⟶ B ∧ A
